#Versión 2.0

Una vez probados los resultados con una red neuronal entrenada desde cero, se va a tratar de mejorar sensiblemente los resultados de la clasificación. El dataset del que se dispone es bastante pequeño para este tipo de entrenamientos (alrededor de 300 imágenes por clase), por lo que se hará uso de la técnica *Transfer Learning*.

Primero, se instalan las dependecias necesarias y se importan los datos:


In [0]:
!pip install tensorflow
!pip install keras==2.3.1


In [0]:
from google.colab import drive
drive.mount('/content/drive/')
!unzip '/content/drive/My Drive/Robot Jardinero/flowers.zip' -d '/content/flower_data'

Después, se importa el VGG16, una red neuronal convolucional propuesta por K. Simonyan y A. Zisserman. Este modelo consigue un 92.7% de precisión en ImageNet, un dataset con más de 14 millones de imágenes pertenecientes a 1000 clases.

In [11]:
import numpy as np
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.optimizers import Adam

train_datagen=ImageDataGenerator(rescale=1./255.,
                                 #rotation_range=40,
                                 #width_shift_range=0.2,
                                 #height_shift_range=0.2,
                                 #shear_range=0.2,
                                 #zoom_range=0.2,
                                 #horizontal_flip=True,
                                 #fill_mode="nearest",
                                 validation_split=0.2,
                                 preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory('/content/flower_data/flowers',
                                                    target_size=(150,150), #to meet VGG requirements
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    subset='training')
validation_generator = train_datagen.flow_from_directory('/content/flower_data/flowers',
                                                         target_size=(150,150),
                                                         color_mode='rgb',
                                                         batch_size=32,
                                                         class_mode='categorical',
                                                         subset='validation')

#Como base, tomaremos el VGG16, con casi 15 millones de parámetros entrenables
base_model=VGG16(weights="imagenet",
                 include_top=False, 
                 input_shape=(150,150,3))


#Se añade una última capa (Fully Connected), que será la que se entrenará con las imágenes
modeloPropio = Sequential()
modeloPropio.add(base_model)
base_model.summary()

modeloPropio.add(Flatten())
modeloPropio.add(Dense(units=4096,activation="relu"))
#modeloPropio.add(BatchNormalization())
modeloPropio.add(Dense(units=2048,activation="relu"))
#modeloPropio.add(BatchNormalization())
modeloPropio.add(Dense(units=1024,activation="relu"))
modeloPropio.add(Dense(units=64,activation="relu"))
modeloPropio.add(Dense(5, activation="softmax"))

modeloPropio.summary()

from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

#Establecemos que el modelo de VGG16 ya está entrenado, por lo que se deshabilita como parte del entrenamiento
base_model.trainable=False

#custom optimizer
sgd = optimizers.SGD(lr=0.01, decay=1e-3, momentum=0.9, nesterov=True)

modeloPropio.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001), metrics=["accuracy"])

#step size
step_size_train = train_generator.n//train_generator.batch_size
step_size_valid = validation_generator.n//validation_generator.batch_size



#fit the model
history = modeloPropio.fit_generator(train_generator,
                                     steps_per_epoch=step_size_train,
                                     epochs=30,
                                     validation_data=validation_generator,
                                     validation_steps=step_size_valid,
                                     shuffle = True)

Found 3462 images belonging to 5 classes.
Found 861 images belonging to 5 classes.
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
____________________________________________

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

# Plot training & validation accuracy values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: ignored

In [0]:
import os
# Guardar el Modelo
modeloPropio.save('/content/drive/My Drive/TF_85.h5')
